In [1]:
import os
import pandas as pd
import pickle
from sklearn.preprocessing import MinMaxScaler
from scripts.collaborative_recommendations import (
    get_recommendation_collaborative,
    measure_distances_indices_collaborative,
)
import numpy as np
import torchvision.transforms as tt
import torch
from evaluate_functions import (
    calculate_mrr,
    calculate_map,
    calculate_ndcg,
    get_table_results,
    get_user_item_preds,
    get_user_images,
)
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from scripts.DNN_predicter import TrainDataset, ResnetEmbeddings, PredictionModel

In [2]:
cwd = os.getcwd()
parent_directory = os.path.abspath(os.path.join(cwd, os.pardir))
data_directory = os.path.join(parent_directory, "data")

In [3]:
df_test = pd.read_json(
    os.path.join(data_directory, "limited_10_3_sweaters_reviews_sentiment_test.json"),
    orient="split",
)
df_train = pd.read_json(
    os.path.join(data_directory, "limited_10_3_sweaters_reviews_sentiment_train.json"),
    orient="split",
)
melted_df_test = pd.melt(
    df_test.reset_index(), id_vars="index", value_vars=df_test.columns
)
melted_df_test.columns = ["user_id", "item_id", "sentiment"]
melted_df_test = melted_df_test[melted_df_test["sentiment"] != -1].reset_index(
    drop=True
)

In [4]:
pictures_dir = os.path.join(data_directory, "sweater_pics")
image_size = 480
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

In [5]:
scaler = MinMaxScaler(feature_range=(0, 1))
melted_df_test["sentiment"] = scaler.fit_transform(
    melted_df_test.sentiment.values.reshape(-1, 1)
)

In [6]:
users_to_analyze = list(set(df_test.index.unique()) & set(df_train.index.unique()))

In [7]:
df_analyze = df_test[df_test.index.isin(users_to_analyze)]

In [8]:
melted_df_analyze = pd.melt(
    df_analyze.reset_index(), id_vars="index", value_vars=df_analyze.columns
)
melted_df_analyze = melted_df_analyze[melted_df_analyze["value"] != -1].reset_index(
    drop=True
)
melted_df_analyze.columns = ["user_id", "item_id", "sentiment"]

# Poniższa tabela informuje nas o przedmiotach które kupili użytkownicy.

In [9]:
positive_sentiments_df = melted_df_analyze[melted_df_analyze["sentiment"] > 0]
user_target_items = (
    positive_sentiments_df.groupby("user_id")["item_id"].unique().to_frame()
)

In [10]:
user_target_items.head()

,item_id
user_id,
A103Y1M1F27V4C,[B00CWDV8VI]
A11JVD2EWWTFNJ,"[B00CMXZHP6, B01FD285DS]"
A135FN638AZOYG,[B01ATZ0DXY]
A13UG665AR8LCV,[B00WIVQ738]
A14T6K6RWUEMVE,[B013HOCKWQ]


# Założenia systemu.
## Dane 
- zostały wykorzystane dane z platformy Amazon. Dane zostały oczyszczone z wartości nieprawidłowych
- każdy przedmiot musiał mieć min. 10 recenzji
- każdy użytkownik wystawił więcej niż 3 recenzje unikatowym produktom
- przedmioty zostały 'odfiltrowane' na podstawie liczby użytkowników, którzy je zakupili
- dane zostały podzielone na treningowe i testowe przed treningiem, aby zapobiec wyciekowi danych
- original: 73675 recenzji, 2684 przedmiotów
- uzyskano dane postaci: 2691 recenzji, 743 przedmiotów 

## System zbudowany sekwencyjnie na podstawie poniższych modalności:
- tekst Review użytkowników na temat zakupionych ubrań
- na podstawie tekstu za pomocą modelu głębokiego uzyskano sentyment każdej z wypowiedzi
- sentyment został zmapowany na wartości liczbowe
- została wygenerowana tabela, gdzie w kolejnych wierszach znajdowały się ID użytkowników, w kolumnach ID przedmiotów, a wartościami był sentyment
- tak tabelę wykorzystano do wyznaczenia najbliższych sąsiadów dla danego użytkownika
- collaboratywna część systemu zwraca zadaną liczbę rekomendacji w oparciu o miarę cosinusową
- kolejnym komponentu systemu jest głęboka sieć neuronowa, której zadaniem jest stworzenie rankingu wśród już zaproponowanych przez część collaboratywną przedmiotów
- wejściem sieci jest ID użytkownika i obrazek przedmiotu, dla którego chcemy poznać wartość prawdopodobieństwa, że użytkownikowi spodoba się dany przedmiot
- przykład uczący wyglądał następująco: user_id, image -> sentiment (w skali od 0 do 1)
- sieć nauczyła się embeddingów użytkowników
- komponentem sieci był ResNet pozbawiony warstwy klasyfikacyjnej
- dla każdego zarekomendowanego z poprzedniego kroku przedmiotu uzyskiwano prawdopodobieństwo, że użytkownikowi spodoba się przedmiot
- sortowano rekomendacje malejąco
- tak uzyskany ranking był rankingiem ostatecznym

# Przebadanie działania części z collaboratywnym filtrowaniem

In [11]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available!")
else:
    device = torch.device("cpu")
    print("CUDA is not available! Using CPU")

CUDA is available!


In [12]:
distances, indices = measure_distances_indices_collaborative(
    df_train, neighbors_to_scoring=20, metric="cosine"
)

In [13]:
user_target_items_sorted = user_target_items.sort_values(
    by="item_id", key=lambda x: x.apply(len), ascending=False
)
n_items = 8

In [14]:
mean_map = calculate_map(
    df=user_target_items_sorted,
    recommendation_function=get_recommendation_collaborative,
    n_items=n_items,
    distances=distances,
    indices=indices,
)
print(f"Mean Average Precision: {mean_map}")

Mean Average Precision: 0.018


In [15]:
mean_mrr = calculate_mrr(
    df=user_target_items_sorted,
    recommendation_function=get_recommendation_collaborative,
    n_items=n_items,
    distances=distances,
    indices=indices,
)
print(f"Mean Reciprocal Rank: {mean_mrr}")

Mean Reciprocal Rank: 0.069


In [16]:
ndcg = calculate_ndcg(
    df=user_target_items_sorted,
    recommendation_function=get_recommendation_collaborative,
    n_items=n_items,
)
print(f"Mean Average Precision: {mean_map}")

Mean Average Precision: 0.018


System opiera się na pierwszej części - collaboratywnej. Jej potencjalnie mierne działanie powoduje, że jakość całego systemu jest obniżona. 

# Trening modelu głębokiego.

In [17]:
n_epochs = 30
lr = 0.001
model = pickle.load(
    open(os.path.join(data_directory, f"model_{n_epochs}_{lr}.pkl"), "rb")
)

model.eval();

In [18]:
log_dir = os.path.join(data_directory, "logs")
os.makedirs(log_dir, exist_ok=True)
writer = SummaryWriter()
test_dataset = TrainDataset(
    df=melted_df_test,
    pictures_dir=pictures_dir,
    transfrom=tt.Compose(
        [
            tt.Resize(image_size),
            tt.CenterCrop(image_size),
            tt.ToTensor(),
            tt.Normalize(*stats),
        ]),
    image_size=image_size,
)
batch_size = 16
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
dataiter = iter(test_loader)
images, labels = next(dataiter)
writer.add_graph(model.cpu(), images)

In [19]:
%load_ext tensorboard
%tensorboard --logdir=runs --host localhost --port 6006

Launching TensorBoard...

!<img src="DNN_architecture.png">

## Training loss

!<img src="training_loss_over_epochs_30.png">

## Training accuracy

!<img src="training_accuracy_over_epochs_30.png">

# Testing loss

!<img src="testing_loss_over_epoch_15.png">

# Testing accuracy
!<img src="testing_accuracy_over_epoch_15.png">

# Połączony system.

In [20]:
users_ids = list(user_target_items_sorted.index)

In [21]:
recommendations = [
    get_recommendation_collaborative(df_train, user_id, n_items, distances, indices)
    for user_id in users_ids
]

In [22]:
user_index_array = []
for user_id in users_ids:
    user_index_array.append(model.df_train.index.get_loc(user_id))

In [23]:
user_images = get_user_images(
    recommendations=recommendations,
    pictures_dir=pictures_dir,
    transform=tt.Compose(
        [
            tt.Resize(image_size),
            tt.CenterCrop(image_size),
            tt.ToTensor(),
            tt.Normalize(*stats),
        ]
    ),
    image_size=image_size,
)

Images processed:   0%|          | 0/321 [00:00<?, ?it/s]

In [24]:
n_users_preds = get_user_item_preds(
    user_index_array=user_index_array,
    user_images=user_images,
    users_ids=users_ids,
    model=model.to(device),
    device=device,
)

Predicter processing:   0%|          | 0/321 [00:00<?, ?it/s]

In [25]:
ranking = {
    user_id: [item[0] for item in item_list]
    for user_id, item_list in n_users_preds.items()
}

ranking_system = pd.DataFrame(list(ranking.items()), columns=["user_id", "item_id"])
ranking_system = ranking_system.set_index("user_id")

In [26]:
calculate_map(
    df=user_target_items_sorted,
    recommendations=ranking_system,
    recommendation_function=None,
    n_items=n_items,
    distances=distances,
    indices=indices,
)

0.018

In [27]:
calculate_mrr(
    df=user_target_items_sorted,
    recommendations=ranking_system,
    recommendation_function=None,
    n_items=n_items,
    distances=distances,
    indices=indices,
)

0.077

In [28]:
calculate_ndcg(
    df=user_target_items_sorted,
    recommendations=ranking_system,
    recommendation_function=None,
    n_items=n_items,
    distances=distances,
    indices=indices,
)

0.1

# Najpopularniejsze przedmioty rekomendowane

In [29]:
scoring = pd.DataFrame(data=np.sum(df_train, axis=0), columns=["score"]).sort_values(
    by="score", ascending=False
)
recs = scoring.index[:5]

In [30]:
most_popular_df = pd.DataFrame(index=ranking_system.index)
most_popular_df["item_id"] = [recs.values] * most_popular_df.shape[0]

In [31]:
calculate_map(
    df=user_target_items_sorted,
    recommendations=most_popular_df,
    recommendation_function=None,
    n_items=n_items,
    distances=distances,
    indices=indices,
)

0.022

In [32]:
calculate_mrr(
    df=user_target_items_sorted,
    recommendations=most_popular_df,
    recommendation_function=None,
    n_items=n_items,
    distances=distances,
    indices=indices,
)

0.074

In [33]:
calculate_ndcg(
    df=user_target_items_sorted,
    recommendations=most_popular_df,
    recommendation_function=None,
    n_items=n_items,
    distances=distances,
    indices=indices,
)

0.097

# Przebadanie jak zachowują się wyniki w zależności od ilości rekomendowanych przedmiotów

In [34]:
results = get_table_results(user_target_items_sorted, ranking_system, None)
df = pd.DataFrame(results).transpose()
df

,map_value,mrr_value,ndcg_value
2,0.029,0.056,0.1
4,0.023,0.068,0.1
6,0.018,0.072,0.1
8,0.018,0.077,0.1
10,0.018,0.077,0.1
12,0.018,0.077,0.1
14,0.018,0.077,0.1
16,0.018,0.077,0.1
18,0.018,0.077,0.1
20,0.018,0.077,0.1


In [35]:
results = get_table_results(user_target_items_sorted, most_popular_df, None)
df = pd.DataFrame(results).transpose()
df

,map_value,mrr_value,ndcg_value
2,0.033,0.062,0.097
4,0.022,0.071,0.097
6,0.022,0.074,0.097
8,0.022,0.074,0.097
10,0.022,0.074,0.097
12,0.022,0.074,0.097
14,0.022,0.074,0.097
16,0.022,0.074,0.097
18,0.022,0.074,0.097
20,0.022,0.074,0.097


In [36]:
results = get_table_results(
    user_target_items_sorted, None, get_recommendation_collaborative
)
df = pd.DataFrame(results).transpose()
df

,map_value,mrr_value,ndcg_value
2,0.037,0.048,0.060
4,0.030,0.064,0.085
6,0.023,0.068,0.093
8,0.018,0.069,0.096
10,0.015,0.070,0.098
12,0.013,0.071,0.099
14,0.011,0.071,0.099
16,0.009,0.071,0.099
18,0.008,0.071,0.100
20,0.008,0.071,0.100


# Cold start
Problem cold startu został rozwiązany za pomocą rekomendowania najpopularniejszych przedmiotów.

# Wnioski

- sekwencyjny model wykorzystujący wiele modalności jest modelem skomplikowanym wymagającym wiele czasu na dostrojenie i dobranie hiperparametrów
- sukces w treningu jednego z komponentów nie zapewnia sukcesu całego systemu
- połączenie wielu modalności może przynieść zysk w postaci informacji o użytkowniku
- ograniczenie zbioru może spowodować okrojenie informacji na temat zbioru